In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

SCOPUS_FILE = "2108_Scopus Results_August 2025.xlsx"
WOS_FILE    = "2508_WoS Results_August 2025.xlsx"
YEAR_MIN, YEAR_MAX = 0, 2025

STAMP = datetime.now().strftime("%Y%m%d_%H%M")
OUTPUT_XLSX = f"SLR_WoS_Scopus_{STAMP}.xlsx"

SCOPUS_COL_MAP = {
    "Title": "Title",
    "Authors": "Authors",
    "Year": "Year",
    "Source title": "Source title",
    "Cited by": "Cited by",
    "DOI": "DOI",
    "Link": "Link",
    "Abstract": "Abstract",
    "Author Keywords": "Author Keywords",
    "Index Keywords": "Index Keywords",
    "Document Type": "Document Type",
}
WOS_COL_MAP = {
    "Article Title": "Title",
    "Authors": "Authors",
    "Publication Year": "Year",
    "Source Title": "Source title",
    "Times Cited, All Databases": "Cited by",
    "DOI": "DOI",
    "DOI Link": "Link",
    "Abstract": "Abstract",
    "Author Keywords": "Author Keywords",
    "Keywords Plus": "Index Keywords",
    "Document Type": "Document Type",
}

EXPECTED_COLS = [
    "Title","Authors","Year","Source title","Cited by","DOI","Link",
    "Abstract","Author Keywords","Index Keywords","Keywords","Database","Document Type"
]

def _ensure_columns(df, expected):
    for c in expected:
        if c not in df.columns:
            df[c] = ""
    return df[expected].copy()

def load_scopus(path: str) -> pd.DataFrame:
    df = pd.read_excel(path, engine="openpyxl")
    df = df.rename(columns=SCOPUS_COL_MAP)

    ak = df.get("Author Keywords", pd.Series("", index=df.index)).fillna("")
    ik = df.get("Index Keywords", pd.Series("", index=df.index)).fillna("")
    df["Keywords"] = (ak + "; " + ik).str.strip("; ")

    df["Year"] = pd.to_numeric(df.get("Year"), errors="coerce")
    df["Cited by"] = pd.to_numeric(df.get("Cited by"), errors="coerce").fillna(0)
    df = df[(df["Year"].isna()) | ((df["Year"] >= YEAR_MIN) & (df["Year"] <= YEAR_MAX))]

    if "Language of Original Document" in df.columns:
        df = df[df["Language of Original Document"].astype(str).str.lower().eq("english")]

    df["Link"] = df.get("Link", "")
    has_no_link = df["Link"].astype(str).str.strip().eq("")
    has_doi = df["DOI"].astype(str).str.strip().ne("")
    df.loc[has_no_link & has_doi, "Link"] = "https://doi.org/" + df.loc[has_no_link & has_doi, "DOI"].astype(str)

    df["Database"] = "Scopus"
    return _ensure_columns(df, EXPECTED_COLS)

def load_wos(path: str) -> pd.DataFrame:
    df = pd.read_excel(path, engine="openpyxl")
    df = df.rename(columns=WOS_COL_MAP)

    ak = df.get("Author Keywords", pd.Series("", index=df.index)).fillna("")
    ik = df.get("Index Keywords", pd.Series("", index=df.index)).fillna("")
    df["Keywords"] = (ak + "; " + ik).str.strip("; ")

    df["Year"] = pd.to_numeric(df.get("Year"), errors="coerce")
    df["Cited by"] = pd.to_numeric(df.get("Cited by"), errors="coerce").fillna(0)
    df = df[(df["Year"].isna()) | ((df["Year"] >= YEAR_MIN) & (df["Year"] <= YEAR_MAX))]

    if "Language" in df.columns:
        df = df[df["Language"].astype(str).str.lower().eq("english")]

    df["Link"] = df.get("Link", "")
    has_no_link = df["Link"].astype(str).str.strip().eq("")
    has_doi = df["DOI"].astype(str).str.strip().ne("")
    df.loc[has_no_link & has_doi, "Link"] = "https://doi.org/" + df.loc[has_no_link & has_doi, "DOI"].astype(str)

    df["Database"] = "WoS"
    return _ensure_columns(df, EXPECTED_COLS)

def deduplicate_doi(df):
    n0 = len(df)
    x = df.copy()
    x["__doi_norm"] = x["DOI"].astype(str).str.strip().str.lower()
    x["__abs_len"] = x["Abstract"].astype(str).str.len()
    x["__has_link"] = (x["Link"].astype(str).str.len() > 0).astype(int)

    with_doi = x[x["__doi_norm"].ne("")].sort_values(
        by=["__doi_norm","__abs_len","__has_link"],
        ascending=[True, False, False]
    ).drop_duplicates(subset="__doi_norm", keep="first")

    without_doi = x[x["__doi_norm"].eq("")]
    out = pd.concat([with_doi, without_doi], ignore_index=True)
    out = out.drop(columns=["__doi_norm","__abs_len","__has_link"])
    return out, {"initial": n0, "after": len(out), "removed": n0 - len(out)}

KEEP_TYPES = [
    "Article", "Review",
    "Article; Data Paper", "Data paper",
    "Article; Book Chapter", "Book chapter",
    "Article; Early Access",
    "Conference paper"
]

ACCESS_TERMS = [
    r"\belectricity\b",
    r"\belectricity access\b",
    r"\benergy access\b",
    r"\belectrification\b",
    r"\belectrification (?:access|rate|level|coverage|mapping|measurement|progress)\b"
]
CORE_TERMS = [
    r"\bmeasur\w*\b", 
    r"\bestimat\w*\b", 
    r"\bmap\w*\b", 
    r"\bdefin\w*\b",
    r"\btrack\w*\b", 
    r"\bsdg tracking\b", r"\btracking sdg ?7\b", "Sustainable Development Goal 7"
r"monitor\w*(?:\W+\w+){0,5}\s+(?:electrification|electricit(?:y)?|energy\s+access)"
]
METHOD_TERMS = [
    # Surveys
    "multi-tier framework", "mtf", "world bank",
    "demographic and health survey", "dhs",
    "living standards measurement study", "lsms",
    "multiple indicator cluster survey", "mics",
    "world health survey", "whs",
    "study on global ageing and adult health", "sage",
    "household survey", "census", "censal",

    # Remote sensing / geospatial
    "nighttime light", "night-time light", "night time light",
    "Visible Infrared Imaging Radiometer Suite", "viirs", "Defense Meteorological Satellite Program", "dmsp", 
    "remote sensing", "geospatial", "spatial",
    "satellite data", "satellite imagery",

    # Mobile data
    "mobile phone data", "call detail record", "cdr",

    # Solar/off-grid

    r"\bsolar home systems?\b",
    r"\bshs\b",
    r"\bsolar (?:pv?s?|photovoltaics?|power|generation|electrification)\b",
    r"\boff[ -]?grid(?: solar)?\b"
    
]

EXCLUDE_TERMS = ["lightning","thunderstorm","volcan","weather radar","precipitation","atmospher", "weather", "weather conditions",
                "optimisation","optimization","techno[- ]?economic","levelized cost of electricity", "lcoe", "net present value", "npv","payback","homer","sizing","cost-?benefit",
                 "battery", "batteries", "electric vehicle", "transport", "transfer",
                 "smart grid", "pollution"
]

VALIDATION_TERMS = [
    r"\bvalidat\w*\b",       
    r"\bcompar(?:e|es|ed|ing|ison|isons)\b"
]


def screen_keywords_simple(df):
    texts = (df["Title"].fillna("") + " " + df["Keywords"].fillna("")).str.lower()

    hit_access   = texts.str.contains("|".join(ACCESS_TERMS),     regex=True)
    hit_core     = texts.str.contains("|".join(CORE_TERMS),       regex=True)
    hit_method   = texts.str.contains("|".join(METHOD_TERMS),     regex=True)
    hit_excl     = texts.str.contains("|".join(EXCLUDE_TERMS),    regex=True)
    hit_validate = texts.str.contains("|".join(VALIDATION_TERMS), regex=True)

    kw_pass = hit_access & (hit_core | hit_method) & ~hit_excl

    out = df.copy()
    out["hit_access"]   = hit_access
    out["hit_core"]     = hit_core
    out["hit_method"]   = hit_method
    out["hit_excl"]     = hit_excl
    out["hit_validate"] = hit_validate
    out["kw_pass"]      = kw_pass
    out["kw_fail"]      = ~kw_pass
    out["kw_reason"]    = np.where(kw_pass, "rule_access_coreOrMethod", "fail")

    out["proceed to abstract check"] = np.where(out["kw_pass"] & out["hit_validate"], "yes", "")

    return out


df_scopus = load_scopus(SCOPUS_FILE)
df_wos = load_wos(WOS_FILE)

df_all = pd.concat([df_scopus, df_wos], ignore_index=True)
raw_total = len(df_all)
print(f"[0] Raw merged total: {raw_total}")

df_all = df_all[df_all["Document Type"].isin(KEEP_TYPES)]
after_doctype = len(df_all)
print(f"[1] After doc-type filter: {after_doctype} (removed {raw_total - after_doctype})")

df_dedup, stats = deduplicate_doi(df_all)
print(f"[2] Dedup: {stats}")

df_kw = screen_keywords_simple(df_dedup)
df_pass = df_kw[df_kw["kw_pass"]].copy()
df_fail = df_kw[~df_kw["kw_pass"]].copy()
print(f"[3] Keyword screening -> Pass: {len(df_pass)} | Fail: {len(df_fail)}")

txt_pass = (df_pass["Title"].fillna("") + " " +
            df_pass["Abstract"].fillna("") + " " +
            df_pass["Keywords"].fillna("")).str.lower()

prox_patterns = [
    r"(?:measur\w*|estimat\w*|map\w*|defin\w*|validat\w*|metric|indicator|index|tracking?)"
    r"(?:\W+\w+){0,8}\b(?:electrification|electricity access|energy access)\b",
    r"\b(?:electrification|electricity access|energy access)\b"
    r"(?:\W+\w+){0,8}(?:measur\w*|estimat\w*|map\w*|defin\w*|validat\w*|metric|indicator|index|tracking?)"
]
design_rx = r"\b(?:optimization|optimisation|feasibility|sizing|homer|lcoe|npv|payback|economic analysis|techno[- ]?economic)\b"
survey_rx = r"\b(?:household survey|dhs|lsms|mics|census|national survey|multi[- ]tier framework|mtf)\b"
rs_rx     = r"\b(?:night[ -]?time light|viirs|dmsp|remote sensing|satellite|geospatial|high resolution imagery)\b"

df_pass["hit_prox8"]   = txt_pass.str.contains("|".join(prox_patterns), regex=True)
df_pass["design_lang"] = txt_pass.str.contains(design_rx, regex=True)
df_pass["survey_hits"] = txt_pass.str.count(survey_rx)
df_pass["rs_hits"]     = txt_pass.str.count(rs_rx)
df_pass["abstract_len"]= df_pass["Abstract"].fillna("").str.len()

year  = pd.to_numeric(df_pass["Year"], errors="coerce").fillna(0)
cites = pd.to_numeric(df_pass["Cited by"], errors="coerce").fillna(0)

def suggest_decision(row):
    if row["hit_prox8"] and (row["survey_hits"] > 0 or row["rs_hits"] > 0):
        return "include"
    if row["design_lang"] and not row["hit_prox8"]:
        return "exclude"
    if row["abstract_len"] < 250:
        return "exclude"
    return "maybe"

df_pass["abstract_decision"] = df_pass.apply(suggest_decision, axis=1)
df_pass["abstract_notes"]    = ""

print("[4] Auto-suggested decisions assigned.")

with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as w:
    df_all.to_excel(w, sheet_name="0_Raw_Merged", index=False)
    df_dedup.to_excel(w, sheet_name="1_Deduplicated", index=False)
    df_kw.to_excel(w, sheet_name="2_KeywordScreen_All", index=False)
    df_pass.to_excel(w, sheet_name="2a_Keyword_PASS", index=False)
    df_fail.to_excel(w, sheet_name="2b_Keyword_FAIL", index=False)

print(f"Done. Saved {OUTPUT_XLSX}")

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

OLD_FILE  = "2608_Dissertation_Systematic Literature Review_v2.xlsx"
OLD_SHEET = "WOS+Scop"
CUR_SHEET = "2a_Keyword_PASS"

MANUAL_COLS = [
    "Include (Yes/No)",
    "Inclusion reason (initial screening)",
    "Exclusion Reason",
    "Ground Truth Mentioned (Yes/No/Maybe)",
    "Second check - Relevance of abstract (Yes/No)",
    "Methodology Type",
    "Validation Strategy",
    "Region/Country",
    "Notes",
    "Highly relevant (Yes/No)",
    "Abstract excerpts",
]

curr = pd.read_excel(OUTPUT_XLSX, sheet_name=CUR_SHEET, engine="openpyxl").copy()
curr["DOI_norm"]   = curr["DOI"].astype(str).str.strip().str.lower()
curr["Title_norm"] = curr["Title"].astype(str).str.strip().str.lower()

old = pd.read_excel(OLD_FILE, sheet_name=OLD_SHEET, engine="openpyxl").copy()
old["DOI_norm"]   = old["DOI"].astype(str).str.strip().str.lower()
old["Title_norm"] = old["Title"].astype(str).str.strip().str.lower()

for c in MANUAL_COLS:
    if c not in old.columns:
        old[c] = ""

manual = old[["DOI_norm","Title_norm"] + MANUAL_COLS].copy()

by_doi   = manual.dropna(subset=["DOI_norm"]).drop_duplicates("DOI_norm").set_index("DOI_norm")
by_title = manual.dropna(subset=["Title_norm"]).drop_duplicates("Title_norm").set_index("Title_norm")

for c in MANUAL_COLS:
    curr[c] = curr["DOI_norm"].map(by_doi[c])  # DOI match
    mask = curr[c].isna() & curr["Title_norm"].notna()
    curr.loc[mask, c] = curr.loc[mask, "Title_norm"].map(by_title[c])  # Title fallback
    curr[c] = curr[c].fillna("")

with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl", mode="a", if_sheet_exists="replace") as w:
    curr.drop(columns=["DOI_norm","Title_norm"], errors="ignore").to_excel(w, sheet_name=CUR_SHEET, index=False)

wb = load_workbook(OUTPUT_XLSX)
ws = wb[CUR_SHEET]
header_cells = {cell.value: cell for cell in next(ws.iter_rows(min_row=1, max_row=1))}
yellow = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
for col_name in MANUAL_COLS:
    cell = header_cells.get(col_name)
    if cell:
        cell.fill = yellow
wb.save(OUTPUT_XLSX)

print("Manual-screening columns merged and highlighted in 2a_Keyword_PASS.")


In [ ]:
must_appear_dois = [
    "10.1038/s41597-019-0122-6",        # Falchetta 2019
    "10.1140/epjds/s13688-022-00371-5", # Pokhriyal 2022
    "10.1109/powerafrica57932.2023.10363249", # Temesgen 2023
    "10.1016/j.rse.2019.111430",        # Stokes & Seto 2019
    "10.1007/s41324-019-00307-8",       # Paul 2020
    "10.1109/jstars.2023.3316310",      # He 2023
    "10.1080/01431161.2017.1420936",    # Dugoua 2018
    "10.1016/j.oneear.2020.03.007",     # Falchetta 2020
    "10.1080/19376812.2021.1897023",    # Dlamini 2022
    "10.1145/3447555.3464871",          # Correa 2021
    "10.3390/ijgi11040222",             # Ren 2022
    "10.1002/9780470979563.ch15",       # Elvidge 2011 (book chapter)
    "10.1016/j.joule.2024.05.001"       # Manual entry 2024
]

import re
def normalize_doi(x: str) -> str:
    s = str(x or "").strip().lower()
    # strip common prefixes
    s = re.sub(r"^(https?://(dx\.)?doi\.org/|doi:\s*)", "", s)
    return s

df_kw = df_kw.copy()
df_kw.loc[:, "DOI_norm"] = df_kw["DOI"].apply(normalize_doi)

passes = df_kw[df_kw["kw_pass"]].copy()

must_norm = {normalize_doi(d) for d in must_appear_dois if d}

present_norm = set(passes["DOI_norm"].dropna().tolist())
missing = sorted(must_norm - present_norm)

if missing:
    print("These must-appear DOIs are missing after filters:", missing)
else:
    print("All must-appear DOIs survived the filters!")


In [ ]:
excluded = df_fail[df_fail["DOI"].astype(str).str.lower().isin([d.lower() for d in must_appear_dois])]
print("Must-appear DOIs excluded:", excluded["DOI"].tolist())